### Definição de Constantes e Parâmetros

In [1]:
import numpy as np

np.set_printoptions(linewidth=200)

# Definição de Constantes e Parâmetros

# Módulo de elasticidade (Pa)
E = 70e9

# Área da seção transversal (m²)
A = 0.000063

# Comprimentos dos elementos (m)
L1 = np.sqrt(5)  # Comprimento do elemento 1
L2 = 2           # Comprimento do elemento 2

# Ângulos dos elementos (radianos)
ang1 = np.arctan(-0.5)  # Ângulo do elemento 1
ang2 = 0                # Ângulo do elemento 2

### Cálculo das Matrizes de Rigidez

In [2]:
# Constantes de rigidez axial dos elementos
EAKL1 = E * A / L1  # Rigidez axial do elemento 1
EAKL2 = E * A / L2  # Rigidez axial do elemento 2

# Matriz de Rigidez
def calcular_matriz_rigidez(E, A, L, angulo):
    EA_L = E * A / L
    c = np.cos(angulo)
    s = np.sin(angulo)
    return EA_L * np.array([
        [c**2, c*s, -c**2, -c*s],
        [c*s, s**2, -c*s, -s**2],
        [-c**2, -c*s, c**2, c*s],
        [-c*s, -s**2, c*s, s**2]
    ])

# Matriz de rigidez do elemento 1
k1 = calcular_matriz_rigidez(E, A, L1, ang1)

# Matriz de rigidez do elemento 2
k2 = calcular_matriz_rigidez(E, A, L2, ang2)


print("Matriz de rigidez do elemento 1 (k1):")
print(k1)

print("\nMatriz de rigidez do elemento 2 (k2):")
print(k2)

Matriz de rigidez do elemento 1 (k1):
[[ 1577769.56492385  -788884.78246193 -1577769.56492385   788884.78246193]
 [ -788884.78246193   394442.39123096   788884.78246193  -394442.39123096]
 [-1577769.56492385   788884.78246193  1577769.56492385  -788884.78246193]
 [  788884.78246193  -394442.39123096  -788884.78246193   394442.39123096]]

Matriz de rigidez do elemento 2 (k2):
[[ 2205000.        0. -2205000.       -0.]
 [       0.        0.       -0.       -0.]
 [-2205000.       -0.  2205000.        0.]
 [      -0.       -0.        0.        0.]]


### Montagem da Matriz Global

In [3]:
def montar_matriz_global(k1, k2):
    # Inicialização da matriz global de rigidez (6x6) com zeros
    matriz_global = np.zeros((6, 6))
    # Inserção da matriz de rigidez do elemento 1 na matriz global
    matriz_global[0:4, 0:4] += k1
    # Inserção da matriz de rigidez do elemento 2 na matriz global
    matriz_global[2:6, 2:6] += k2
    return matriz_global

matriz_global = montar_matriz_global(k1, k2)

print("Matriz global de rigidez:")
print(matriz_global)

Matriz global de rigidez:
[[ 1577769.56492385  -788884.78246193 -1577769.56492385   788884.78246193        0.                0.        ]
 [ -788884.78246193   394442.39123096   788884.78246193  -394442.39123096        0.                0.        ]
 [-1577769.56492385   788884.78246193  3782769.56492385  -788884.78246193 -2205000.                0.        ]
 [  788884.78246193  -394442.39123096  -788884.78246193   394442.39123096        0.                0.        ]
 [       0.                0.         -2205000.                0.          2205000.                0.        ]
 [       0.                0.                0.                0.                0.                0.        ]]


### Resolução do Sistema

In [4]:
# Redução da matriz global para considerar as condições de contorno
# Como sabemos que os deslocamentos na parede são nulos, então cancelamos as linhas e colunas correspondentes, ficando assim apenas uma matriz reduzida
matriz_reduzida = matriz_global[2:4, 2:4]

# Vetor de forças reduzido (considerando apenas os graus de liberdade desconhecidos)
vetor_forcas_reduzido = np.array([0, -1000])  # Força aplicada no nó correspondente

# Resolvendo o sistema de equações para os deslocamentos nodais desconhecidos
deslocamentos = np.linalg.solve(matriz_reduzida, vetor_forcas_reduzido)

# Exibindo os deslocamentos nodais desconhecidos
print("Deslocamentos nodais desconhecidos:")
print(deslocamentos)

Deslocamentos nodais desconhecidos:
[-0.00090703 -0.00434928]


### Cálculo das Forças de Reação

In [5]:
# Reconstrução do vetor de deslocamentos completo, incluindo os deslocamentos conhecidos (zeros nos apoios)
deslocamentos_completos = np.zeros(6)
deslocamentos_completos[2:4] = deslocamentos

# Cálculo das forças de reação nos apoios utilizando a matriz global de rigidez
forcas_reacao = np.dot(matriz_global, deslocamentos_completos)

# Exibindo as forças de reação nos apoios
print("Forças de reação nos apoios:")
print(forcas_reacao[0:2])  # Reações no nó 0
print(forcas_reacao[2:4])  # Reações no nó 1
print(forcas_reacao[4:6])  # Reações no nó 2

Forças de reação nos apoios:
[-2000.  1000.]
[    0. -1000.]
[2000.    0.]


### Transformação para Coordenadas Locais

In [6]:
# Matriz de transformação para o elemento 1
T1 = np.array([
    [np.cos(ang1), np.sin(ang1), 0, 0],
    [-np.sin(ang1), np.cos(ang1), 0, 0],
    [0, 0, np.cos(ang1), np.sin(ang1)],
    [0, 0, -np.sin(ang1), np.cos(ang1)]
])

# Matriz de transformação para o elemento 2
T2 = np.array([
    [np.cos(ang2), np.sin(ang2), 0, 0],
    [-np.sin(ang2), np.cos(ang2), 0, 0],
    [0, 0, np.cos(ang2), np.sin(ang2)],
    [0, 0, -np.sin(ang2), np.cos(ang2)]
])

# Transformação dos deslocamentos globais para coordenadas locais
deslocamentos_locais_1 = np.dot(T1, deslocamentos_completos[0:4])
deslocamentos_locais_2 = np.dot(T2, deslocamentos_completos[2:6])

# Exibindo os deslocamentos transformados para coordenadas locais
print("Deslocamentos em coordenadas locais para o elemento 1:")
print(deslocamentos_locais_1)

print("\nDeslocamentos em coordenadas locais para o elemento 2:")
print(deslocamentos_locais_2)

Deslocamentos em coordenadas locais para o elemento 1:
[ 0.          0.          0.00113379 -0.00429575]

Deslocamentos em coordenadas locais para o elemento 2:
[-0.00090703 -0.00434928  0.          0.        ]


### Cálculo das Forças Axiais

In [7]:
# Matriz de forças-deslocamentos local para cálculo das forças axiais
matriz_forcas_deslocamento = np.array([
    [1, 0, -1, 0],
    [0, 0, 0, 0],
    [-1, 0, 1, 0],
    [0, 0, 0, 0]
])

# Cálculo das forças axiais no elemento 1
forcas_axiais_1 = EAKL1 * np.dot(matriz_forcas_deslocamento, deslocamentos_locais_1)
print("Forças axiais no elemento 1:")
print(forcas_axiais_1)

# Cálculo das forças axiais no elemento 2
forcas_axiais_2 = EAKL2 * np.dot(matriz_forcas_deslocamento, deslocamentos_locais_2)
print("\nForças axiais no elemento 2:")
print(forcas_axiais_2)

Forças axiais no elemento 1:
[-2236.0679775     0.         2236.0679775     0.       ]

Forças axiais no elemento 2:
[-2000.     0.  2000.     0.]


### Cálculo das Tensões

In [8]:
# Cálculo das tensões no elemento 1
tensao_1 = (forcas_axiais_1[2] + forcas_axiais_1[3]) / A
print("Tensão no elemento 1 (Pa):")
print(tensao_1)

# Cálculo das tensões no elemento 2
tensao_2 = (forcas_axiais_2[0] + forcas_axiais_2[1]) / A
print("\nTensão no elemento 2 (Pa):")
print(tensao_2)

Tensão no elemento 1 (Pa):
35493142.49999667

Tensão no elemento 2 (Pa):
-31746031.746031743


# Otimização

In [9]:
# Cálculo do momento de Inércia da seção transversal
I = (0.012**4-0.009**4)/12
print(I)
# Cálculo da força axial de compressão crítica
Pcr = ((np.pi**2)*E*I)/(L2**2)
print(Pcr)

1.1812500000000002e-09
204.02322847876908


Agora, calcularemos a força axial de compressão crítica com o coeficiente de segurança $1,2$

In [10]:
N = np.abs(forcas_axiais_2[0]*1.2)
print(N)

2399.9999999999995


$$P_{cr} = \frac{\pi^2EI}{L^2}$$
$$2400 = \frac{\pi^2\times 70 \times 10^9 \times I}{2^2}$$

In [11]:
I = (2400 * L2**2) / (np.pi**2 * E)

print(f"O valor de I é: {I:.2e} m^4")

O valor de I é: 1.39e-08 m^4


$$I = \frac{a^4_e - a^4_i}{12} $$
Reforçarei $a_e$ e manterei $a_i$
$$I = \frac{a^4_e - 0,009^4}{12} $$
Calculando $a_e$

In [12]:
# Calculando a_e
a_e = (12 * I + 0.009**4)**0.25
print(f"O valor de a_e é: {a_e} m")

O valor de a_e é: 0.02040346138097511 m


### Tensão de escoamento


A tensão de escoamento com coeficiente de segurança $2$ é dada por

In [13]:
Sy = tensao_1 * 2
print(Sy)

70986284.99999334


Como podemos ver, a tensão 

$$$$